# week 9

In [2]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 27.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


question 1

In [3]:
import numpy as np
import statsmodels.api as sm

def simulate(A=1, B=1, C=10, D=1000):
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

def test_power(A=1, B=1, C=10, D=1000, reps=5000):
    count = 0
    for _ in range(reps):
        Y, X, W = simulate(A, B, C, D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        t_value = model.tvalues[1]   # t-value for X (first non-constant)
        if abs(t_value) > 1.96:
            count += 1
    return count / reps

prob = test_power(A=1, B=1, C=10, D=1000, reps=5000)
print(f"Probability of detecting nonzero effect ≈ {prob:.3f}")

Probability of detecting nonzero effect ≈ 0.882


question 2

In [4]:
from scipy.stats import skew

def simulate(A=1, B=1, C=10, D=1000):
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

def estimate_coeffs(A=1, B=1, C=10, D=1000, reps=5000):
    betas = []
    for _ in range(reps):
        Y, X, W = simulate(A, B, C, D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        betas.append(model.params[1])  # coefficient for X
    return np.array(betas)

betas = estimate_coeffs(A=1, B=1, C=10, D=1000, reps=5000)
print("Skew of estimate:", skew(betas))


Skew of estimate: 0.0008569642516143284


question 3

In [5]:
def simulate(A=1, B=1, C=10, D=1000):
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

def power_for_B(B, reps=2000):
    count = 0
    for _ in range(reps):
        Y, X, W = simulate(A=1, B=B, C=10, D=1000)
        XW = sm.add_constant(np.column_stack([X, W]))
        tval = sm.OLS(Y, XW).fit().tvalues[1]   # t for X
        if abs(tval) > 1.96:
            count += 1
    return count / reps

for B in [0.2, 0.6, 1.8, 5.4]:
    p = power_for_B(B, reps=2000)
    print(f"B={B:4.1f} -> power ≈ {p:.3f}")


B= 0.2 -> power ≈ 0.096
B= 0.6 -> power ≈ 0.475
B= 1.8 -> power ≈ 1.000
B= 5.4 -> power ≈ 1.000


question 4

In [6]:
def simulate(A=1, B=1, C=10, D=100): # different D
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

def power_for_A(A, reps=5000, B=1, C=10, D=100, seed=None):
    if seed is not None:
        np.random.seed(seed)
    count = 0
    for _ in range(reps):
        Y, X, W = simulate(A, B, C, D)
        XW = sm.add_constant(np.column_stack([X, W]))
        tval = sm.OLS(Y, XW).fit().tvalues[1]  # t for X
        if abs(tval) > 1.96:
            count += 1
    return count / reps

for A in [1.0, 4.0, 2.0, 0.5]:
    p = power_for_A(A, reps=5000)
    print(f"A={A:3.1f} -> power ≈ {p:.4f}")


A=1.0 -> power ≈ 0.1682
A=4.0 -> power ≈ 0.9694
A=2.0 -> power ≈ 0.5102
A=0.5 -> power ≈ 0.0844
